In [ ]:
https://cdn.zephyrcms.com/2348096f-8890-41c7-b890-bcab8364e607/-/inline/yes/cip-35-testing-compressive-strength-of-concrete.pdf
https://www.bigdreadymix.com/what-is-the-standard-strength-of-concrete/#:~:text=Usually%2C%20the%20compressive%20strength%20of,10%2C000%20psi%20(70%20MPa).
https://cor-tuf.com/everything-you-need-to-know-about-concrete-strength/#:~:text=Compressive%20strength%20is%20tested%20by,Testing%20%26%20Materials)%20standard%20C39.
https://archive.ics.uci.edu/ml/datasets/concrete+compressive+strength
https://civilread.com/find-quantity-cement-sand-aggregate/
https://civiconcepts.com/blog/concrete-mix-ratio#:~:text=One%20of%20the%20best%20ratios,%2C%20steps%2C%20and%20foundation%20walls.
https://sci-hub.se/10.1016/s0008-8846(98)00165-3
https://www.kaggle.com/datasets/sinamhd9/concrete-comprehensive-strength
https://www.analyticsvidhya.com/blog/2021/04/concrete-strength-prediction-using-machine-learning-with-python-code/
https://medium.com/data-science-at-microsoft/introduction-to-gaussian-process-regression-part-2-application-to-predicting-concrete-strength-3facef04f639#0dd7

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor as GBR, RandomForestRegressor as RFR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly
from plotly import tools
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/regression/concrete_strength/Concrete_Data.csv')
df

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [ ]:
pd.DataFrame({'Dtype': df.dtypes, 'Nunique': df.nunique(), 'Isnull': df.isnull().sum()}, index=df.columns)

,Dtype,Nunique,Isnull
Cement (component 1)(kg in a m^3 mixture),float64,278,0
Blast Furnace Slag (component 2)(kg in a m^3 mixture),float64,185,0
Fly Ash (component 3)(kg in a m^3 mixture),float64,156,0
Water (component 4)(kg in a m^3 mixture),float64,195,0
Superplasticizer (component 5)(kg in a m^3 mixture),float64,111,0
Coarse Aggregate (component 6)(kg in a m^3 mixture),float64,284,0
Fine Aggregate (component 7)(kg in a m^3 mixture),float64,302,0
Age (day),int64,14,0
"Concrete compressive strength(MPa, megapascals)",float64,845,0


In [ ]:
df.columns

Index(['Cement (component 1)(kg in a m^3 mixture)',
       'Blast Furnace Slag (component 2)(kg in a m^3 mixture)',
       'Fly Ash (component 3)(kg in a m^3 mixture)',
       'Water  (component 4)(kg in a m^3 mixture)',
       'Superplasticizer (component 5)(kg in a m^3 mixture)',
       'Coarse Aggregate  (component 6)(kg in a m^3 mixture)',
       'Fine Aggregate (component 7)(kg in a m^3 mixture)', 'Age (day)',
       'Concrete compressive strength(MPa, megapascals) '],
      dtype='object')

In [ ]:
out_column = 'Concrete compressive strength(MPa, megapascals) '
to_remove = []
features = [c for c in df.columns if (c != out_column) and (c not in to_remove)]
categorical = [c for c in features if (df[c].dtype=='object') and (df[c].nunique() > 2)]
binary = [c for c in features if df[c].nunique() == 2]
numerical = [col for col in features if col not in categorical + binary]
df[numerical] = df[numerical].apply(pd.to_numeric,1)
fig = go.Figure(data=go.Heatmap(z=df[numerical+[out_column]].corr(),x=numerical+[out_column],y=numerical+[out_column]))
fig.show()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df[features], df[out_column], test_size=0.2, random_state=42)
Y_train, Y_test = Y_train.values.ravel() + 1e-15, Y_test.values.ravel() + 1e-15

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((824, 8), (206, 8), (824,), (206,))

In [ ]:
hp = [1.e+02, 1.e-05, 1.e-05, 7.e+00]
hp = [np.round(hp[0]).astype(int), hp[1], hp[2], np.round(hp[3]).astype(int)]
regr = RFR(n_estimators=hp[0], n_jobs=-1, min_samples_split=hp[1], min_samples_leaf=hp[2], max_features=hp[3], bootstrap=True, random_state=1)
regr.fit(X_train, Y_train)

RandomForestRegressor(max_features=7, min_samples_leaf=1e-05,
                      min_samples_split=1e-05, n_jobs=-1, random_state=1)

In [ ]:
hp = [1.00000000e+02, 1.00000000e-05, 2.56676833e-05, 3.61885250e-01,8.00000000e+00]
hp = [np.round(hp[0]).astype(int), hp[1], hp[2], hp[3], np.round(hp[4]).astype(int)]
regr = GBR(n_estimators=hp[0], min_samples_split=hp[1], min_samples_leaf=hp[2], learning_rate=hp[3], max_features=hp[4], random_state=1)
regr.fit(X_train, Y_train)

GradientBoostingRegressor(learning_rate=0.36188525, max_features=8,
                          min_samples_leaf=2.56676833e-05,
                          min_samples_split=1e-05, random_state=1)

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=1)
score = cross_val_score(regr, X_train, Y_train, cv=kf, scoring='neg_mean_absolute_error',).mean()
score

-3.075801027171599

In [ ]:
train_pred = regr.predict(X_train)
test_pred = regr.predict(X_test)

eval_df = pd.DataFrame([{'R2': r2_score(Y_train, train_pred), 'MSE': mean_squared_error(Y_train, train_pred, squared=False), 'MAE': mean_absolute_error(Y_train, train_pred)},
                        {'R2': r2_score(Y_test, test_pred), 'MSE': mean_squared_error(Y_test, test_pred, squared=False), 'MAE': mean_absolute_error(Y_test, test_pred)}], index=['TRAIN', 'TEST'])
eval_df

,R2,MSE,MAE
TRAIN,0.980090,2.378287,1.671364
TEST,0.911416,4.777652,3.329124


In [ ]:
pd.DataFrame({'True': Y_test, 'Pred': test_pred})

,True,Pred
0,52.91,52.602515
1,55.90,47.626881
2,74.50,75.092193
3,35.30,34.323270
4,10.54,11.151845
...,...,...
201,56.14,56.254128
202,18.75,21.317049
203,38.00,47.455021
204,74.36,69.637627


In [ ]:
from sklearn.inspection import permutation_importance

result = permutation_importance(regr, X_test, Y_test, n_repeats=10, random_state=42, n_jobs=2, scoring='r2')
fig = go.Figure()
for i in range(X_test.shape[1]):
    fig.add_trace(go.Box(x=result['importances'][i], name=X_test.columns[i]))

fig.show()

In [ ]:
import warnings
import traceback
import dask
import numpy as np
import pandas as pd
import umap
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor as RFR, GradientBoostingRegressor as GBR
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
import category_encoders as ce
import optimization.numpy_version.single_objective.continuous as co

def _cv_score(hp):
    hp = [np.round(hp[0]).astype(int), hp[1], hp[2], np.round(hp[3]).astype(int)]
    regr = RFR(n_estimators=hp[0], n_jobs=-1, min_samples_split=hp[1], min_samples_leaf=hp[2], max_features=hp[3], bootstrap=True, random_state=1)
    score = cross_val_score(regr, X_train, Y_train, cv=kf,scoring='neg_mean_absolute_error').mean()
    return hp, -score

def cv_score(hp):
    hp = [np.round(hp[0]).astype(int), hp[1], hp[2], hp[3], np.round(hp[4]).astype(int)]
    regr = GBR(n_estimators=hp[0], min_samples_split=hp[1], min_samples_leaf=hp[2], learning_rate=hp[3], max_features=hp[4], random_state=1)
    score = cross_val_score(regr, X_train, Y_train, cv=kf, scoring='neg_mean_absolute_error',).mean()
    return hp, -score

def objf(pop, dim=None):
    population, fitness = [], []
    for hp in pop:
        _hp, mse = dask.delayed(cv_score, nout=2)(hp)
        population.append(_hp), fitness.append(mse)
    population, fitness = dask.compute(population, fitness)
    population, fitness = np.asarray(population), np.asarray(fitness)
    return population, fitness  # duration =  27.99079418182373

df = pd.read_csv('/home/skyolia/Téléchargements/Concrete_Data.csv')
########################################################################################################################
out_column = 'Concrete compressive strength(MPa, megapascals) '
to_remove = []
features = [c for c in df.columns if (c != out_column) and (c not in to_remove)]
categorical = [c for c in features if (df[c].dtype=='object') and (df[c].nunique() > 2)]
binary = [c for c in features if df[c].nunique() == 2]
numerical = [col for col in features if col not in categorical + binary]
df[numerical] = df[numerical].apply(pd.to_numeric,1)
########################################################################################################################
X_train, X_test, Y_train, Y_test = train_test_split(df[features], df[out_column], test_size=0.2, random_state=42)
Y_train, Y_test = Y_train.values.ravel() + 1e-15, Y_test.values.ravel() + 1e-15

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
########################################################################################################################
def ccf(population):
    return population

kf = KFold(n_splits=10, shuffle=True, random_state=1)

#lb, ub = [2, 1e-5, 1e-5, 1], [100, 1, 0.5, X_train.shape[1]]
lb, ub = [2, 1e-5, 1e-5, 1e-5, 1], [100, 1, 0.5, 2, X_train.shape[1]]
dim = len(lb)
optimizer = co.CPSO(objf, ccf, dim, lb, ub, 50, 5000)